# 2/ Introduction to SQL AI Function: generating fake data with custom Model Serving Endpoint

For this demo, we'll start by generating fake data using `AI_QUERY()`. 

The sample data will mimics customer reviews for grocery products submitted to an e-commerce website.

## Working with `AI_QUERY` function

Our function signature is the following:

```
SELECT ai_query(endpointName, request) for external models and foundation models. 
SELECT ai_query(endpointName, request, returnType) for custom model serving endpoint. 
```

`AI_QUERY` will send the prompt to the remote model configured and retrive the result as SQL.

*Note: this will reproduce the behavior or the built-in `gen_ai` function, but leveraging one of the Model Serving Endpoint of your choice.*<br/>
*If you're looking at quickly generating data, we recommend you to just go with the built-in.*

*This notebook will use the foundation Dbrx Instruct model for inference*

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=DBSQL&org_id=1221206873714816&notebook=%2F02-Generate-fake-data-with-AI-functions&demo_name=sql-ai-functions&event=VIEW&path=%2F_dbdemos%2FDBSQL%2Fsql-ai-functions%2F02-Generate-fake-data-with-AI-functions&version=1">

To run this notebook, connect to <b> SQL endpoint </b>. The AI_QUERY function is available on Databricks SQL Pro and Serverless.

In [0]:
%sql
-- as previously, make sure you run this notebook using a SQL Warehouse (not a cluster)
SELECT assert_true(current_version().dbsql_version is not null, 'YOU MUST USE A SQL WAREHOUSE');

USE CATALOG main;
CREATE SCHEMA IF NOT EXISTS dbdemos_ai_query;
USE SCHEMA dbdemos_ai_query;

"assert_true((current_version().dbsql_version IS NOT NULL), YOU MUST USE A SQL WAREHOUSE)"
null


In [0]:
%sql
SELECT
  AI_QUERY(
    "databricks-dbrx-instruct",
    "Generate a short product review for a red dress. The customer is very happy with the article."
  ) as product_review

product_review
"""I am thrilled with my purchase of this red dress! The color is vibrant and eye-catching, and the fit is flattering and comfortable. I have received numerous compliments while wearing it. Highly recommend!"""


### Introduction to SQL Function: adding a wrapper function to simplify the call

While it's easy to call this function, having to our model endpoint name as parameter can be harder to use, especially for Data Analyst who should focus on crafting proper prompt. 

To simplify our demo next steps, we'll create a wrapper SQL function `ASK_LLM_MODEL` with a string as input parameter (the question to ask) and wrap all the model configuration.

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/sql-ai-functions/sql-ai-query-function-review-wrapper.png" width="1200px">

In [0]:
%sql
CREATE OR REPLACE FUNCTION ASK_LLM_MODEL(prompt STRING) 
  RETURNS STRING
  RETURN 
    AI_QUERY("databricks-dbrx-instruct", prompt);

-- ALTER FUNCTION ASK_LLM_MODEL OWNER TO `your_principal`; -- for the demo only, make sure other users can access your function

In [0]:
%sql
SELECT ASK_LLM_MODEL("Generate a short product review for a red dress. The customer is very happy with the article.")

main.dbdemos_ai_query.ask_llm_model(Generate a short product review for a red dress. The customer is very happy with the article.)
"""I am thrilled with my purchase of this red dress! The color is vibrant and eye-catching, and the fit is flattering and comfortable. I have received numerous compliments while wearing it. Highly recommend!"""


## Generating a more complete sample dataset with prompt engineering

Now that we know how to send a basic query to Open AI using SQL functions, let's ask the model a more detailed question.

We'll directly ask to model to generate multiple rows and directly return as a json. 

Here's a prompt example to generate JSON:
```
Generate a sample dataset for me of 2 rows that contains the following columns: "date" (random dates in 2022), 
"review_id" (random id), "product_name" (use popular grocery product brands), and "review". Reviews should mimic useful product reviews 
left on an e-commerce marketplace website. 

The reviews should vary in length (shortest: one sentence, longest: 2 paragraphs), sentiment, and complexity. A very complex review 
would talk about multiple topics (entities) about the product with varying sentiment per topic. Provide a mix of positive, negative, 
and neutral reviews

Return JSON ONLY. No other text outside the JSON. JSON format:
[{"review_date":<date>, "review_id":<review_id>, "product_name":<product_name>, "review":<review>}]
```

In [0]:
%sql
SELECT ASK_LLM_MODEL(
      'Generate a sample dataset of 2 rows that contains the following columns: "date" (random dates in 2022), 
      "review_id" (random id), "customer_id" (random long from 1 to 100)  and "review". Reviews should mimic useful product reviews 
      left on an e-commerce marketplace website. 
      
      The reviews should be about a popular grocery brands product

      The reviews should vary in length (shortest: one sentence, longest: 2 paragraphs), sentiment, and complexity. A very complex review 
      would talk about multiple topics (entities) about the product with varying sentiment per topic. Provide a mix of positive, negative, 
      and neutral reviews.

      Give me JSON only. No text outside JSON. No explanations or notes
      [{"review_date":<date>, "review_id":<long>, "customer_id":<long>, "review":<string>}]') as fake_reviews;

fake_reviews
"[ {""review_date"": ""2022-01-01"", ""review_id"": 12345, ""customer_id"": 50, ""review"": ""The product was great, I really enjoyed it.""}, {""review_date"": ""2022-06-15"", ""review_id"": 67890, ""customer_id"": 25, ""review"": ""I was disappointed with my purchase. The quality was lacking and the taste was off.""} ]"


## Converting the results as json 

Our results are looking good. All we now have to do is transform the results from text as a JSON and explode the results over N rows.

Let's create a new function to do that:

In [0]:
%sql
CREATE OR REPLACE FUNCTION GENERATE_FAKE_REVIEWS(num_reviews INT DEFAULT 5)
  RETURNS array<struct<review_date:date, review_id:long, customer_id:long, review:string>>
  RETURN 
  SELECT FROM_JSON(
      ASK_LLM_MODEL(
        CONCAT('Generate a sample dataset of ', num_reviews, ' rows that contains the following columns: "date" (random dates in 2022), 
        "review_id" (random long), "customer_id" (random long from 1 to 100) and "review". 
        Reviews should mimic useful product reviews from popular grocery brands product left on an e-commerce marketplace website. The review must include the product name.

        The reviews should vary in length (shortest: 5 sentence, longest: 10 sentences).
        Provide a mix of positive, negative, and neutral reviews but mostly negative.

        Give me JSON only. No text outside JSON. No explanations or notes
        [{"review_date":<date>, "review_id":<long>, "customer_id":<long>, "review":<string>}]')), 
        "array<struct<review_date:date, review_id:long, customer_id:long, review:string>>")

-- ALTER FUNCTION GENERATE_FAKE_REVIEWS OWNER TO `your_principal`; -- for the demo only, make sure other users can access your function

In [0]:
%sql
SELECT
  review.*
FROM
  (
    SELECT
      explode(reviews) as review
    FROM
      (
        SELECT
          GENERATE_FAKE_REVIEWS(10) as reviews
      )
  )

review_date,review_id,customer_id,review
2022-01-01,123456789012345,42,I was really disappointed with my purchase of Kellogg's Frosted Flakes. The box was only half full and the cereal was stale. I expect better quality from such a well-known brand.
2022-02-20,987654321098765,73,"I'm a big fan of Coca-Cola, but the last bottle I bought tasted off. It was flat and had a strange aftertaste. I hope this was just a one-time issue."
2022-03-15,456123789456123,19,"I've been buying Quaker Oats for years, but the last batch I got was inedible. The oats were hard and didn't cook properly. I'm not sure what happened, but I'm not happy."
2022-04-10,951753456987451,88,"I'm a fan of Frito-Lay chips, but the bag I bought last week was mostly air. I felt ripped off. I expect more chips in my bag."
2022-05-05,741852963012369,34,"I've always loved Nutella, but the jar I bought last month had a weird texture. It was grainy and not as smooth as it usually is. I hope this was just a fluke."
2022-06-22,369258147258147,61,"I'm a fan of Campbell's soup, but the can I bought last week was dented and the soup tasted off. I'm not sure if it was because of the dent or not, but I was disappointed."
2022-07-08,852963012369852,5,"I've been buying Cheerios for years, but the box I got last week was stale. I'm not sure if it was old or what, but I was not happy."
2022-08-20,258147258147258,97,"I'm a fan of Lay's potato chips, but the bag I bought last week was mostly broken pieces. I felt like I didn't get my money's worth."
2022-09-15,963012369852963,28,"I've always loved Skippy peanut butter, but the jar I bought last month had a strange taste. It was almost bitter. I'm not sure what happened, but I was not happy."
2022-10-10,741258147258147,82,"I'm a fan of Kraft mac and cheese, but the box I bought last week was missing the cheese packet. I was not happy. I expect better quality from such a well-known brand."



## Saving our dataset as a table to be used directly in our demo.

*Note that if you want to create more rows, you can first create a table and add multiple rows, with extra information that you can then concatenate to your prompt like categories, expected customer satisfaction etc. Once your table is created you can then call a new custom GENERATE function taking more parameters and crafting a more advanced prompt*

In [0]:
%sql
CREATE
OR REPLACE TABLE fake_reviews COMMENT "Raw Review Data" AS
SELECT
  review.*
FROM
  (
    SELECT
      explode(reviews) as review
    FROM
      (
        SELECT
          GENERATE_FAKE_REVIEWS(10) as reviews
      )
  )

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE FUNCTION GENERATE_FAKE_CUSTOMERS(num_reviews INT DEFAULT 10)
  RETURNS array<struct<customer_id:long, firstname:string, lastname:string, order_count:int>>
  RETURN 
  SELECT FROM_JSON(
      ASK_LLM_MODEL(
        CONCAT('Generate a sample dataset of ', num_reviews, ' customers containing the following columns: 
        "customer_id" (long from 1 to ', num_reviews, '), "firstname", "lastname" and order_count (random positive number, smaller than 200)

        Give me JSON only. No text outside JSON. No explanations or notes
        [{"customer_id":<long>, "firstname":<string>, "lastname":<string>, "order_count":<int>}]')), 
        "array<struct<customer_id:long, firstname:string, lastname:string, order_count:int>>")
        
-- ALTER FUNCTION GENERATE_FAKE_CUSTOMERS OWNER TO `your_principal`; -- for the demo only, make sure other users can access your function

In [0]:
%sql
CREATE OR REPLACE TABLE fake_customers
  COMMENT "Raw customers"
  AS
  SELECT customer.* FROM (
    SELECT explode(customers) as customer FROM (
      SELECT GENERATE_FAKE_CUSTOMERS(10) as customers))

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM fake_reviews

review_date,review_id,customer_id,review
2022-01-01,123456789012345,42,I was really disappointed with my purchase of Kellogg's Frosted Flakes. The box was only half full and the cereal was stale. I expect better quality from such a well-known brand.
2022-02-20,987654321098765,73,"I'm a big fan of Coca-Cola, but the last bottle I bought tasted off. It was flat and had a strange aftertaste. I hope this was just a one-time issue."
2022-03-15,456123789456123,19,"I've been buying Quaker Oats for years, but the last batch I got was inedible. The oats were hard and didn't cook properly. I'm not sure what happened, but I'm not happy."
2022-04-10,951753456987451,88,"I'm a fan of Frito-Lay chips, but the bag I bought last week was mostly air. I felt ripped off. I expect more chips in my bag."
2022-05-05,741852963012369,34,"I've always loved Nutella, but the jar I bought last month had a weird texture. It was grainy and not as smooth as it usually is. I hope this was just a fluke."
2022-06-22,369258147258147,61,"I'm a fan of Campbell's soup, but the can I bought last week was dented and the soup tasted off. I'm not sure if it was because of the dent or not, but I was disappointed."
2022-07-08,852963012369852,5,"I've been buying Cheerios for years, but the box I got last week was stale. I'm not sure if it was old or what, but I was not happy."
2022-08-20,258147258147258,97,"I'm a fan of Lay's potato chips, but the bag I bought last week was mostly broken pieces. I felt like I didn't get my money's worth."
2022-09-15,963012369852963,28,"I've always loved Skippy peanut butter, but the jar I bought last month had a strange taste. It was almost bitter. I'm not sure what happened, but I was not happy."
2022-10-10,741258147258147,82,"I'm a fan of Kraft mac and cheese, but the box I bought last week was missing the cheese packet. I was really disappointed. I expect better quality from such a well-known brand."


In [0]:
%sql
SELECT * FROM fake_customers

customer_id,firstname,lastname,order_count
1,John,Doe,123
2,Jane,Doe,45
3,Mike,Johnson,67
4,Amy,Smith,89
5,Sarah,Brown,150
6,Peter,Davis,111
7,Emily,Miller,12
8,Ryan,Williams,34
9,Lisa,Taylor,56
10,Daniel,Thomas,78


## Next steps
We're now ready to implement our pipeline to extract information from our reviews! Open [03-automated-product-review-and-answer]($./03-automated-product-review-and-answer) to continue.


Go back to [the introduction]($./00-SQL-AI-Functions-Introduction)